In [1]:
import pickle
import pandas as pd
import folium
import geopy
import geopandas
from tqdm import tqdm
import os
import warnings
warnings.filterwarnings("ignore")
import re

In [2]:
from geopy import Nominatim
locator = Nominatim(user_agent="myGeocoder")
location = locator.geocode("Champ de Mars, Paris, France")

# import data
I need to drop the `name` column cause there are some names missing.

In [3]:
os.chdir('..')
os.chdir('data')
os.chdir('pickle')
os.getcwd()

'e:\\Cursos\\00ironhack_DA\\00LABS\\vuelve-conmigo-a-italia\\data\\pickle'

In [4]:
path = os.getcwd()
files = os.listdir(path)
files = [f for f in files if f[-6:] == 'pickle']
files

['dict_booking_0.pickle',
 'dict_booking_1.pickle',
 'dict_booking_2.pickle',
 'dict_booking_3.pickle',
 'dict_booking_4.pickle',
 'dict_booking_5.pickle']

In [5]:
list_dicts = []
for file in files:
    with open(f'{file}', 'rb') as dict_booking:
        dict_booking = pickle.load(dict_booking)
        if len(dict_booking['url']) != len(dict_booking['name']):
            del dict_booking['url'][-1]
        print(len(dict_booking['url']), len(dict_booking['name']), len(dict_booking['pagina']), len(dict_booking['currentPrice']), len(dict_booking['address']))
        list_dicts.append(pd.DataFrame(dict_booking))

49 49 49 49 49
1905 1905 1905 1905 1905
1172 1172 1172 1172 1172
75 75 75 75 75
807 807 807 807 807
51 51 51 51 51


In [6]:
df_booking = pd.concat(list_dicts).reset_index(drop=True)
df_booking.head(1)

,url,name,pagina,address,currentPrice,previousPrice,rating,reviewNumber,checkingDate,image
0,https://www.booking.com/hotel/it/b-amp-b-donna...,B&B Donna Teresa,\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n...,"[via giuseppe garibaldi, 137/b, 92100 , 92100 ...",€ 76,€ 124,9.2,525 reviews,2023-09-01,https://cf.bstatic.com/xdata/images/hotel/squa...


In [7]:
df_booking = df_booking.explode('address')
df_booking.head(1)

,url,name,pagina,address,currentPrice,previousPrice,rating,reviewNumber,checkingDate,image
0,https://www.booking.com/hotel/it/b-amp-b-donna...,B&B Donna Teresa,\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n...,"via giuseppe garibaldi, 137/b, 92100 , 92100 a...",€ 76,€ 124,9.2,525 reviews,2023-09-01,https://cf.bstatic.com/xdata/images/hotel/squa...


cleaning duplicated prices 

In [8]:
df_booking['currentPrice'].value_counts()

€ 85          80
€ 90          79
€ 72          74
€ 82          73
€ 100         73
              ..
€ 207          1
€ 191          1
€ 364          1
€ 172€ 138     1
€ 493          1
Name: currentPrice, Length: 414, dtype: int64

In [9]:
def countEuro(column):
    if column.count('€') == 1:
        return 1
    elif column.count('€') == 2:
        return 2
    else:
        return 0

In [10]:
df_booking['cleanPrice'] = df_booking['currentPrice'].apply(countEuro)

In [11]:
df_booking['cleanPrice'].value_counts()

1    4283
2     238
Name: cleanPrice, dtype: int64

In [12]:
df_booking_limpio = df_booking[df_booking['cleanPrice'] == 1]
df_booking_sucio = df_booking[df_booking['cleanPrice'] == 2]

In [13]:
df_booking_sucio.head(1)

,url,name,pagina,address,currentPrice,previousPrice,rating,reviewNumber,checkingDate,image,cleanPrice
17,https://www.booking.com/hotel/it/residence-gui...,Residence Guidaloca,\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n...,"località guidaloca - scopello, 91014 scopello,...",€ 141€ 127,NaN,8.3,624 reviews,2023-09-01,https://cf.bstatic.com/xdata/images/hotel/squa...,2


In [14]:
df_booking_sucio[[0,1,2]] = df_booking_sucio['currentPrice'].str.split('€', expand=True, n = 2)
df_booking_sucio.head(1)

,url,name,pagina,address,currentPrice,previousPrice,rating,reviewNumber,checkingDate,image,cleanPrice,0,1,2
17,https://www.booking.com/hotel/it/residence-gui...,Residence Guidaloca,\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n...,"località guidaloca - scopello, 91014 scopello,...",€ 141€ 127,NaN,8.3,624 reviews,2023-09-01,https://cf.bstatic.com/xdata/images/hotel/squa...,2,,141,127


In [15]:
df_booking_sucio['currentPrice'] = '€ ' + df_booking_sucio[2] 
df_booking_sucio['previousPrice'] = '€ ' + df_booking_sucio[1]

In [37]:
df_booking_sucio.drop(['cleanPrice', 0, 1, 2], axis = 1, inplace=True)
df_booking_limpio.drop(['cleanPrice'], axis = 1, inplace=True)
df_booking_sucio.head(1)

KeyError: "['cleanPrice', 0, 1, 2] not found in axis"

In [17]:
df_booking = pd.concat([df_booking_limpio, df_booking_sucio])
print(df_booking.shape)
df_booking.head(1)

(4521, 11)


,url,name,pagina,address,currentPrice,previousPrice,rating,reviewNumber,checkingDate,image,cleanPrice
0,https://www.booking.com/hotel/it/b-amp-b-donna...,B&B Donna Teresa,\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n...,"via giuseppe garibaldi, 137/b, 92100 , 92100 a...",€ 76,€ 124,9.2,525 reviews,2023-09-01,https://cf.bstatic.com/xdata/images/hotel/squa...,1.0


cleaning info -- parkings and other amenities
- comida
- ensuite bathroom
- piscina
- air conditioning

In [24]:
#parking
df_booking['parking'] = df_booking['pagina'].str.lower().str.findall(r'(free parking)')
df_booking = df_booking.explode('parking').drop_duplicates()
df_booking['parking'].value_counts()

free parking    2582
Name: parking, dtype: int64

In [27]:
#air conditioning
df_booking['air_conditioning'] = df_booking['pagina'].str.lower().str.findall(r'(air conditioning)')
df_booking = df_booking.explode('air_conditioning').drop_duplicates()
df_booking['air_conditioning'].value_counts()

air conditioning    4438
Name: air_conditioning, dtype: int64

In [28]:
#private bathroom
df_booking['bathroom'] = df_booking['pagina'].str.lower().str.findall(r'(private bathroom)')
df_booking = df_booking.explode('bathroom').drop_duplicates()
df_booking['bathroom'].value_counts()

private bathroom    4389
Name: bathroom, dtype: int64

In [30]:
#free wifi
df_booking['wifi'] = df_booking['pagina'].str.lower().str.findall(r'(free wifi)')
df_booking = df_booking.explode('wifi').drop_duplicates()
df_booking['wifi'].value_counts()

free wifi    4440
Name: wifi, dtype: int64

In [31]:
#breakfast
df_booking['breakfast'] = df_booking['pagina'].str.lower().str.findall(r'(breakfast)')
df_booking = df_booking.explode('breakfast').drop_duplicates()
df_booking['breakfast'].value_counts()

breakfast    4024
Name: breakfast, dtype: int64

In [32]:
#coffee machine
df_booking['coffee_tea_maker'] = df_booking['pagina'].str.lower().str.findall(r'(tea/coffee maker)')
df_booking = df_booking.explode('coffee_tea_maker').drop_duplicates()
df_booking['coffee_tea_maker'].value_counts()

tea/coffee maker    1832
Name: coffee_tea_maker, dtype: int64

In [35]:
#laundry
df_booking['laundry'] = df_booking['pagina'].str.lower().str.findall(r'(laundry)')
df_booking = df_booking.explode('laundry').drop_duplicates()
df_booking['laundry'].value_counts()

laundry    2423
Name: laundry, dtype: int64

In [36]:
df_booking.head(1)

,url,name,pagina,address,currentPrice,previousPrice,rating,reviewNumber,checkingDate,image,cleanPrice,parking,air_conditioning,bathroom,wifi,breakfast,coffee_tea_maker,washing_machine,laundry
0,https://www.booking.com/hotel/it/b-amp-b-donna...,B&B Donna Teresa,\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n...,"via giuseppe garibaldi, 137/b, 92100 , 92100 a...",€ 76,€ 124,9.2,525 reviews,2023-09-01,https://cf.bstatic.com/xdata/images/hotel/squa...,1.0,free parking,air conditioning,private bathroom,free wifi,breakfast,tea/coffee maker,NaN,laundry


In [ ]:
print(df_booking.shape)
df_booking.drop_duplicates(inplace=True)
print(df_booking.shape)

checking dtypes

In [ ]:
df_booking.dtypes

In [ ]:
type(df_booking['address'][0])

# Geocoding

In [ ]:
from geopy.extra.rate_limiter import RateLimiter
tqdm.pandas()

# 1 - conveneint function to delay between geocoding calls
geocode = RateLimiter(locator.geocode, min_delay_seconds=1)
# 2- - create location column
df_booking['location'] = df_booking['address'].progress_apply(geocode)
# 3 - create longitude, laatitude and altitude from location column (returns tuple)
df_booking['point'] = df_booking['location'].progress_apply(lambda loc: tuple(loc.point) if loc else None)
# 4 - split point column into latitude, longitude and altitude columns
df_booking[['latitude', 'longitude', 'altitude']] = pd.DataFrame(df_booking['point'].tolist(), index=df_booking.index)

In [ ]:
map1 = folium.Map(
    location=[37.75120776952038, 14.992361985738308],
    tiles='cartodbpositron',
    zoom_start=12,
)
df_booking[df_booking['latitude'].isnull() == False].apply(lambda row:folium.CircleMarker(location=[row["latitude"], row["longitude"]]).add_to(map1), axis=1)
map1

In [ ]:
df_booking.head(1)

In [ ]:
df_booking.iloc[0,0]

revisa el formato de los precios, no está cogiendo bien los valores

In [ ]:
df_booking['url_clean'] = df_booking['url'].str.split('?', expand=True)[0]
df_booking.head(1)

In [ ]:
df_booking.drop(['url', 'pagina'], axis = 1).to_csv('../data/booking.csv', sep='|', encoding='utf-8', index = False)